In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import cv2

In [2]:
def create_base_network(input_shape):
    input = tf.keras.Input(shape=input_shape)
    x = layers.Conv2D(64, (3, 3), activation='relu')(input)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(128, activation='relu')(x)
    return models.Model(input, x)

In [3]:
input_shape = (105, 105, 1)
base_network = create_base_network(input_shape)

In [4]:
input_a = tf.keras.Input(shape=input_shape)
input_b = tf.keras.Input(shape=input_shape)

In [5]:
processed_a = base_network(input_a)
processed_b = base_network(input_b)

In [6]:
distance = layers.Lambda(lambda embeddings: tf.keras.backend.abs(embeddings[0] - embeddings[1]))([processed_a, processed_b])
outputs = layers.Dense(1, activation='sigmoid')(distance)
model = models.Model([input_a, input_b], outputs)


In [7]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
def load_image(filepath):
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (105, 105))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=-1)
    return img

In [9]:
# Prepare your pairs of images and labels for training
def create_pairs(base_image_path, other_image_paths, same_class_paths):
    base_image = load_image(base_image_path)
    pairs = []
    labels = []

    for other_image_path in other_image_paths:
        other_image = load_image(other_image_path)
        pairs.append([base_image, other_image])
        labels.append(0)  # Different class

    for same_class_path in same_class_paths:
        same_class_image = load_image(same_class_path)
        pairs.append([base_image, same_class_image])
        labels.append(1)  # Same class

    return np.array(pairs), np.array(labels)

In [10]:
base_image_path = r"C:\Users\Astha Agarwal\OneDrive\Desktop\learnerSpace_ML\week4\srk1.png"
other_image_paths = [r"C:\Users\Astha Agarwal\OneDrive\Desktop\learnerSpace_ML\week4\modi1.png", r"C:\Users\Astha Agarwal\OneDrive\Desktop\learnerSpace_ML\week4\modi2.png"]
same_class_paths = [r"C:\Users\Astha Agarwal\OneDrive\Desktop\learnerSpace_ML\week4\srk2.png", r"C:\Users\Astha Agarwal\OneDrive\Desktop\learnerSpace_ML\week4\srk2.png"]

pairs, labels = create_pairs(base_image_path, other_image_paths, same_class_paths)
pairs = [pairs[:, 0], pairs[:, 1]]

In [11]:
pairs, labels = create_pairs(base_image_path, other_image_paths, same_class_paths)
pairs = [pairs[:, 0], pairs[:, 1]]

In [12]:
model.fit(pairs, labels, epochs=10, batch_size=32)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.2500 - loss: 0.6916
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step - accuracy: 0.7500 - loss: 0.6320
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - accuracy: 1.0000 - loss: 0.4371
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step - accuracy: 1.0000 - loss: 0.2541
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step - accuracy: 1.0000 - loss: 0.1852
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - accuracy: 1.0000 - loss: 0.1738
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step - accuracy: 1.0000 - loss: 0.1727
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step - accuracy: 1.0000 - loss: 0.1725
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step - accuracy: 1.0000 - loss: 0.1724
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - accuracy: 1.0000 - loss: 0.1723


In [13]:
img1 = load_image(r"C:\Users\Astha Agarwal\OneDrive\Desktop\learnerSpace_ML\week4\srk1.png")
img2 = load_image(r"C:\Users\Astha Agarwal\OneDrive\Desktop\learnerSpace_ML\week4\modi2.png")
prediction = model.predict([np.expand_dims(img1, axis=0), np.expand_dims(img2, axis=0)])
print(f'Prediction: {prediction}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
Prediction: [[1.3696201e-11]]


In [14]:
val_other_image_paths = [r"C:\Users\Astha Agarwal\OneDrive\Desktop\learnerSpace_ML\week4\modi1.png", r"C:\Users\Astha Agarwal\OneDrive\Desktop\learnerSpace_ML\week4\modi2.png"]
val_same_class_paths = [r"C:\Users\Astha Agarwal\OneDrive\Desktop\learnerSpace_ML\week4\srk1.png", r"C:\Users\Astha Agarwal\OneDrive\Desktop\learnerSpace_ML\week4\srk2.png"]

val_pairs, val_labels = create_pairs(base_image_path, val_other_image_paths, val_same_class_paths)
val_pairs = [val_pairs[:, 0], val_pairs[:, 1]]

# Evaluate the model
loss, accuracy = model.evaluate(val_pairs, val_labels)

print(f'Validation Accuracy: {accuracy}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step - accuracy: 1.0000 - loss: 0.1721
Validation Accuracy: 1.0
